In [3]:
import numpy as np
import pandas as pd

In [7]:
p = 0.0
error = {}
count = 0
for dataset in [1, 2]:
    error[dataset] = {}
    for fold_num in range(5):
        error[dataset][fold_num] = {}
        for appliance in ['hvac', 'fridge','dr', 'dw', 'mw']:
            error[dataset][fold_num][appliance] = {}
            for cell_type in ['GRU', 'RNN', 'LSTM']:
                error[dataset][fold_num][appliance][cell_type] = {}
                for hidden_size  in [20, 50, 100]:
                    error[dataset][fold_num][appliance][cell_type][hidden_size] = {}
                    for num_layers in range(1,5):
                        error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers] = {}
                        for bidirectional in [True, False]:
                            error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers][bidirectional] = {}
                            for lr in [0.01, 0.1, 1.0]:
                                error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers][bidirectional][lr] = {}
                                for iters in [1000, 2000, 3000]:
                                    try:
                                        error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers][bidirectional][lr][iters] = \
                                        np.load('../code/baseline/rnn-nested-cv/valid-error-{}-{}-{}-{}-{}-{}-{}-{}-{}-[\'{}\'].npy'.format(fold_num, \
                                                                                                                           dataset, 
                                                                                                                           cell_type, 
                                                                                                                           hidden_size, 
                                                                                                                           num_layers, 
                                                                                                                           bidirectional, 
                                                                                                                           lr, 
                                                                                                                           iters, 
                                                                                                                           p, appliance)).item()

                                    except:
                                        count += 1
                                        print (fold_num, cell_type, hidden_size, num_layers, bidirectional, lr, iters, 0, appliance)
print (count)

0


In [12]:
p = 0.0
best_params = {}
count = 0
for dataset in [1, 2]:
    best_params[dataset] = {}
    for fold_num in range(5):
        best_params[dataset][fold_num] = {}
        for appliance in ['hvac', 'fridge','dr', 'dw', 'mw']:
            best_params[dataset][fold_num][appliance] = {}
            min_error = np.inf
            for cell_type in ['GRU', 'RNN', 'LSTM']:
                for hidden_size  in [20, 50, 100]:
                    for num_layers in range(1,5):
                        for bidirectional in [True, False]:
                            for lr in [0.01, 0.1, 1.0]:
                                for iters in [1000, 2000, 3000]:
                                    if error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers][bidirectional][lr][iters][appliance] < min_error:
                                        min_error = error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers][bidirectional][lr][iters][appliance]
                                        best_params[dataset][fold_num][appliance]['cell_type'] = cell_type
                                        best_params[dataset][fold_num][appliance]['hidden_size'] = hidden_size
                                        best_params[dataset][fold_num][appliance]['num_layers'] = num_layers
                                        best_params[dataset][fold_num][appliance]['bidirectional'] = bidirectional
                                        best_params[dataset][fold_num][appliance]['lr'] = lr
                                        best_params[dataset][fold_num][appliance]['iters'] = iters
print (count)

0


In [19]:
best_params[dataset][1][appliance]

{'bidirectional': True,
 'cell_type': 'RNN',
 'hidden_size': 100,
 'iters': 1000,
 'lr': 0.01,
 'num_layers': 2}

In [16]:
best_params = np.load("../code/baseline/best_param_nested_rnn.npy").item()

In [8]:
err_contri = {'hvac':0.5028749252, 'fridge':0.1129756393, 'dr':0.2837048817, 'dw':0.0653719782,'mw':0.03507257561}

In [23]:
for dataset in [1]:
    for fold_num in range(5):
        print ("fold", fold_num)
        for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
            
            cell_type = best_params[dataset][fold_num][appliance]['cell_type']
            hidden_size = best_params[dataset][fold_num][appliance]['hidden_size'] 
            num_layers = best_params[dataset][fold_num][appliance]['num_layers']
            bidirectional = best_params[dataset][fold_num][appliance]['bidirectional'] 
            lr = best_params[dataset][fold_num][appliance]['lr']
            iters = best_params[dataset][fold_num][appliance]['iters']
            
            e = error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers][bidirectional][lr][iters][appliance]
            
            print (appliance, e, e/err_contri[appliance])

fold 0
hvac 98.5602081648 195.993483122
fridge 23.0447516233 203.979829334
dr 73.5619796993 259.290496725
dw 13.616785021 208.296970596
mw 10.0716479815 287.165906877
fold 1
hvac 99.916129004 198.68982126
fridge 23.704817473 209.822379584
dr 67.4833896823 237.864746203
dw 13.4501938939 205.748613768
mw 7.58962531165 216.39771758
fold 2
hvac 105.419328651 209.633297205
fridge 24.2153095556 214.340982761
dr 69.5197817248 245.042599578
dw 14.4292951778 220.725998739
mw 7.65113499476 218.151500473
fold 3
hvac 107.817836565 214.402888596
fridge 24.1582291356 213.835737379
dr 71.848154455 253.249623427
dw 12.3282676135 188.586424229
mw 7.30655389153 208.326698694
fold 4
hvac 111.874448312 222.469728963
fridge 23.113641215 204.589603194
dr 73.6277456535 259.522307873
dw 14.8799727236 227.620046591
mw 5.33779441023 152.192826372


In [24]:
import itertools

In [26]:
list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw']))

[('hvac', 'fridge', 'dr', 'dw', 'mw'),
 ('hvac', 'fridge', 'dr', 'mw', 'dw'),
 ('hvac', 'fridge', 'dw', 'dr', 'mw'),
 ('hvac', 'fridge', 'dw', 'mw', 'dr'),
 ('hvac', 'fridge', 'mw', 'dr', 'dw'),
 ('hvac', 'fridge', 'mw', 'dw', 'dr'),
 ('hvac', 'dr', 'fridge', 'dw', 'mw'),
 ('hvac', 'dr', 'fridge', 'mw', 'dw'),
 ('hvac', 'dr', 'dw', 'fridge', 'mw'),
 ('hvac', 'dr', 'dw', 'mw', 'fridge'),
 ('hvac', 'dr', 'mw', 'fridge', 'dw'),
 ('hvac', 'dr', 'mw', 'dw', 'fridge'),
 ('hvac', 'dw', 'fridge', 'dr', 'mw'),
 ('hvac', 'dw', 'fridge', 'mw', 'dr'),
 ('hvac', 'dw', 'dr', 'fridge', 'mw'),
 ('hvac', 'dw', 'dr', 'mw', 'fridge'),
 ('hvac', 'dw', 'mw', 'fridge', 'dr'),
 ('hvac', 'dw', 'mw', 'dr', 'fridge'),
 ('hvac', 'mw', 'fridge', 'dr', 'dw'),
 ('hvac', 'mw', 'fridge', 'dw', 'dr'),
 ('hvac', 'mw', 'dr', 'fridge', 'dw'),
 ('hvac', 'mw', 'dr', 'dw', 'fridge'),
 ('hvac', 'mw', 'dw', 'fridge', 'dr'),
 ('hvac', 'mw', 'dw', 'dr', 'fridge'),
 ('fridge', 'hvac', 'dr', 'dw', 'mw'),
 ('fridge', 'hvac', 'dr',